In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();
using JLD2
const resfile = joinpath(@__DIR__, "q3.jld2")
const isautograder = @isdefined autograder;

In [48]:
import Pkg; Pkg.add("MeshCat")

# A Falling Brick: Solving Quadratic Programs (40 pts)
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \lambda \\ \text{ where } M = mI, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\lambda \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ h \begin{bmatrix} \frac{1}{m} J^T \lambda_{k+1} - g \\ v_{k+1} \end{bmatrix} $$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\lambda_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\lambda_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (a): QP formulation (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + v_{k+1}^T M (hg - v_k) \\
    &\text{subject to} && J(q_k + h v_{k+1}) \ge 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown.

## Part (b): Implement an Augmented Lagrangian QP solver (25 pts)
Now that we've shown that we can formulate the falling brick problem as a QP, write an augmented Lagrangian QP solver.

We've provided the following data structure for storing the problem data for a generic QP of the form:
$$ \begin{align}
    &\text{minimize}_{x} && \frac{1}{2} x^T P x + q^T x \\
    &\text{subject to} && A x = b \\
    &&& C x \leq d \\
\end{align} $$

We've also provided a handful of functions that you may find useful when implementing your augmented Lagrangian solver. You're not required to use them.

In [5]:
using Random, LinearAlgebra
"""
    QPData

Holds the data for a Quadratic Program (QP) of the following form:

min 0.5 x'P*x + q'x
st. A*x = b
    C*x ≤ d

# Constructors
    QPData(P,q,A,b,C,d)
    QPData(n::Int,m::Int,p::Int)

The second constructor will initialize all the problem with zeros of the appropriate dimension
"""
struct QPData
    P::Matrix{Float64}
    q::Vector{Float64}
    A::Matrix{Float64}
    b::Vector{Float64}
    C::Matrix{Float64}
    d::Vector{Float64}
end
function QPData(n::Int, m::Int, p::Int)
    QPData(zeros(n,n), zeros(n), zeros(m,n), zeros(m), zeros(p,n), zeros(p))
end
Base.size(qp::QPData) = (length(qp.q), num_eq(qp), num_ineq(qp))
num_eq(qp::QPData) = length(qp.b)
num_ineq(qp::QPData) = length(qp.d)

objective(qp::QPData, x) = 0.5 * x'qp.P*x + qp.q'x
ceq(qp::QPData, x) = qp.A * x - qp.b
cin(qp::QPData, x) = qp.C * x - qp.d

function primal_residual(qp::QPData, x, λ, μ)
    qp.P*x + qp.q + qp.A'λ + qp.C'μ
end

function dual_residual(qp::QPData, x, λ, μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    return [g; max.(0, h)]
end

function complimentarity(qp::QPData, x, λ, μ)
    return [min.(0, μ); μ .* cin(qp, x)]
end

complimentarity (generic function with 1 method)

Implement the following function, which solves the QP specified by a `QPData` object. See the code below for an example of using the `QPData` type and how we expect it to be passed into the method. You're not required to solve a problem with equality constraints (since the brick problem doesn't require it), but we recommend adding in the functionality so you have a fully-functioning QP solver you can use for other problems.

As we saw in class, an augmented Lagrangian solver consists of two loops: an "inner" loop that takes Newtons steps on the unconstrained augmented Lagrangian, and an "outer" loop that updates the penalty parameter and the estimates of the dual variables. We've provided you some starter code below to help you out. If you want to change those other methods (maybe to use a custom Julia type or take in extra input arguments), you're welcome to do so. We'll only call the outer `solve_qp` method from our test scripts.

In [38]:
# TASK: Implement the following method (25 pts)
"""
    solve_qp(qp::QPData, x0, [λ0, μ0]; kwargs...)

Solve the quadratic program (QP) specified by `qp::QPData`, given initial guess `x` for the primal variables, 
and optionally the Lagrange multipliers for the equality `λ` and inequality `μ` constraints.

Returns the optimized solution of primal and dual variables, `xstar,λstar,μstar`.

# Optional Keyword Arguments
* `penalty_initial` initial value of the penalty parameter
* `penalty_scaling` geometric scaling factor for the penalty updates
* `eps_primal` tolerance for primal feasiblity (constraint violation)
* `eps_inner` tolerance for inner Newton solve
* `max_iters` maximum number of outer loop iterations
"""
function solve_qp(qp::QPData, x0, λ0=zeros(num_eq(qp)), μ0=zeros(num_ineq(qp)); 
        penalty_initial=10.0, 
        penalty_scaling=10.0, 
        eps_primal=1e-6,
        eps_inner=1e-6,
        max_iters=20
    )
    x = copy(x0)
    λ = copy(λ0)
    μ = copy(μ0)
    
    ρ = penalty_initial
    ϕ = penalty_scaling
    
    # Start outer loop
    for i = 1:max_iters
        # Solve the inner, unconstrained problem
        x = newton_solve(qp, x, λ, μ, ρ, eps_inner=eps_inner)
        
        # Use the new solution to update the dual variables
        λ, μ = dual_update(qp, x, λ, μ, ρ)
        
        # TODO: update the penalty parameter
        ρ *= ϕ 
        
        if norm(dual_residual(qp, x, λ, μ)) < eps_primal
            # Return the optimized variables
            return x, λ, μ
        end        
    end
    
    @warn "Outer loop max iterations"
    return x, λ, μ 
end


# Optional Methods you may find useful
"""
    newton_solve(qp, x, λ, μ, ρ; kwargs...)

Minimize the augmented Lagranginan given the current values of the dual variables `λ` and `μ` and the penalty parameter `ρ`.
"""
function newton_solve(qp, x, λ, μ, ρ; eps_inner=1e-6)
    for i = 1:10
        # Compute the gradient and the Hessian of the augmented Lagrangian
        r = algrad(qp, x, λ, μ, ρ)
        if norm(r) < eps_inner
            return x
        end
        H = alhess(qp, x, λ, μ, ρ)
        delta = -H\r
        x += delta
        # TODO: Compute the Newton step
        #       A line search will help with convergence, but shouldn't be necessary for our problem since we're providing a good guess each time
    end
    @warn "Inner solve max iterations"
    return x
end

"""
    algrad(qp, x, λ, μ, ρ)

Compute the gradient of the augmented Lagrangian, provided the QP data `qp`, penalty parameter `ρ`,
primal variables `x`, equality Lagrange multipliers `λ` and inequality Lagrange multipliers `μ`
"""
function algrad(qp, x, λ, μ)
    # TODO: compute the gradient of the augmented Lagrangian
    # HINT: be sure to compute the active constraints!
    grad = zero(x)
    grad = qp.P*x + qp.q + qp.A'*λ+ρ*qp.A'*(qp.A*x-qp.b)
    ineq = qp.C*x-qp.d
    for i = 1:length(μ)
        if ineq[i]<0&&μ[i]==0
            grad = grad
        else
            grad += qp.C[i:i,:]'*μ[i]+ρ*qp.C[i:i,:]'*ineq[i]
        end
    end
    return grad
end


"""
    alhess(qp, x, λ, μ, ρ)

Compute the Hessian of the augmented Lagrangian, provided the QP data `qp`, penalty parameter `ρ`,
primal variables `x`, equality Lagrange multipliers `λ` and inequality Lagrange multipliers `μ`
"""
function alhess(qp, x, λ, μ,ρ)
    # TODO: compute the Hessian of the augmented Lagrangian
    n = size(x)
    #hess = Matrix(I,n,n)
    hess = qp.P+ρ*qp.A'*qp.A
    ineq = qp.C*x-qp.d
    for i = 1:length(μ)
        if ineq[i]<0&&μ[i]==0
            hess = hess
        else
            hess += ρ*qp.C[i:i,:]'*qp.C[i:i,:]
        end
    end
    return hess
end

"""
    dual_update(qp, x, λ, μ, ρ)

Update the dual variables `λ` and `μ` give the primal variables `x`, QP data `qp` and penalty parameter `ρ`.
"""
function dual_update(qp, x, λ, μ, ρ)
    # TODO: compute the new values for λ and μ
    λnext = copy(λ)
    μnext = copy(μ)
    λnext += ρ*(qp.A*x-qp.b)
    μnext = max.(0,μnext+ρ*(qp.C*x-qp.d))
    return λnext, μnext
end


dual_update

You can use the following code to test your QP solver.

In [39]:
using Test, Random
Random.seed!(2)
# Setting up and solving a random QP
n,m,p = 10,0,15 
qp = QPData(n,m,p)
P = rand(n,n)
qp.P .= P'P   # make it P.S.D
qp.q .= randn(n)
qp.A .= randn(m,n)
qp.b .= randn(m)
qp.C .= randn(p,n)
qp.d .= randn(p)

# Initial guess
x = randn(n)

# Solve
xstar, λstar, μstar = solve_qp(qp, x)

# Check optimality conditions
@test norm(primal_residual(qp, xstar, λstar, μstar)) < 1e-3
@test norm(dual_residual(qp, xstar, λstar, μstar)) < 1e-6
@test norm(complimentarity(qp, xstar, λstar, μstar)) < 1e-3;

In [40]:
@testset "3b" begin  # POINTS = 25
    Random.seed!(2)
    # Setting up and solving a random QP
    n,m,p = 10,0,15 
    qp = QPData(n,m,p)
    P = rand(n,n)
    qp.P .= P'P   # make it P.S.D
    qp.q .= randn(n)
    qp.A .= randn(m,n)
    qp.b .= randn(m)
    qp.C .= randn(p,n)
    qp.d .= randn(p)

    # Initial guess
    x = randn(n)

    # Solve
    xstar, λstar, μstar = solve_qp(qp, x)
    
    # Check optimality conditions
    @test norm(primal_residual(qp, xstar, λstar, μstar)) < 1e-3  # POINTS = 5
    @test norm(dual_residual(qp, xstar, λstar, μstar)) < 1e-6    # POINTS = 5
    @test norm(complimentarity(qp, xstar, λstar, μstar)) < 1e-3  # POINTS = 5
    
    # Compare with OSQP
    using OSQP, SparseArrays
    model = OSQP.Model()
    OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,p)], u=[qp.b; qp.d],
        eps_abs=1e-6, eps_rel=1e-6, verbose=false)
    res = OSQP.solve!(model)
    @test norm(res.x - xstar) < 1e-3           # POINTS = 5
    @test norm(res.y - [λstar; μstar]) < 1e-3  # POINTS = 5
end;

Test Summary: | Pass  Total
3b            |    5      5


## Part (c): Simulate the system (10 pts)
Use your solver from the previous question to simulate the brick for 3 seconds, from the initial condition of `q0 = [0,1]`, `v0 = [1,0]` with `h=0.01` sec and `m=1`.
Use the provided visualization code to visualize your results.

**NOTE**: If you are unable to get your QP solver to work, feel free to use OSQP to solve the QP. An example of setting up and solving a QP with OSQP is provided above.

In [44]:
# TASK: Implement the following method (2 pts)
"""
    build_qp(q, v; mass=1, h=0.01)

Build the Quadratic Program corresponding to the falling brick example of mass `mass`, 
given the 2D position `q` and velocity `v`, and the time step `h`.

Should return a `QPData` object with the correct sizes.
"""
function build_qp(q,v; mass=1, h=0.01)
    # TODO: finish the function
    Pmat = zeros(2,2)
    qvec = zeros(2)
    A = zeros(0,2)
    b = zeros(0)
    C = zeros(1,2)
    d = zeros(1)
    J = [0 1]
    C = -J*h
    d = J*q
    Pmat = [mass 0
             0 mass]
    vec = [0
            0.81*h] - v
    qvec = Pmat * vec
    # Return as a QPData type
    QPData(Pmat,qvec,A,b,C,d)
end

build_qp

In [45]:
@testset "3c" begin                                # POINTS = 10
    @testset "build qp" begin                      # POINTS = 2
        q = [1.2,-0.36]
        v = [10,-1.2]
        qp = build_qp(q, v)
        @test qp.P ≈ load(resfile, "P") atol=1e-6  # POINTS = 0.5
        @test qp.q ≈ load(resfile, "q") atol=1e-6  # POINTS = 0.5
        @test qp.A ≈ load(resfile, "A") atol=1e-6  # POINTS = 0.25
        @test qp.b ≈ load(resfile, "b") atol=1e-6  # POINTS = 0.25
        @test qp.C ≈ load(resfile, "C") atol=1e-6  # POINTS = 0.25
        @test qp.d ≈ load(resfile, "d") atol=1e-6  # POINTS = 0.25
    end
end;

build qp: Test Failed at In[45]:7
  Expression: ≈(qp.q, load(resfile, "q"), atol = 1.0e-6)
   Evaluated: [-10.0, 1.2081] ≈ [-10.0, 1.2981] (atol=1.0e-6)
Stacktrace:
 [1] macro expansion
   @ In[45]:7 [inlined]
 [2] macro expansion
   @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Test\src\Test.jl:1151 [inlined]
 [3] macro expansion
   @ In[45]:3 [inlined]
 [4] macro expansion
   @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Test\src\Test.jl:1151 [inlined]
 [5] top-level scope
   @ In[45]:2
Test Summary: | Pass  Fail  Total
3c            |    5     1      6
  build qp    |    5     1      6


LoadError: [91mSome tests did not pass: 5 passed, 1 failed, 0 errored, 0 broken.[39m

In [56]:
# TASK: Implement the following method (8 pts)
function simulate_brick(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0
    
   
    # TODO: Simulate the brick by solving the QP
    #  TIP: remember to update your QP after each step
    
    for i = 1:Int(T/h)
        qp = build_qp(qs[i],vs[i];mass=m,h=h)
        x,λ,μ=solve_qp(qp,vs[i])
        vs[i+1] = [x[1],x[2]]
        qs[i+1] = qs[i] + h*vs[i+1]
    end
    # Return the state and velocity trajectories
    return qs, vs
end


simulate_brick (generic function with 3 methods)

### Visualize the Results
Use the following code to visualize the the results of your simulation

In [57]:
# Set up Visualizer
using MeshCat
using GeometryBasics, Colors, CoordinateTransformations
if !isautograder
    vis = Visualizer()
    setobject!(vis["brick"], Rect3D(Vec(0,0,0f0), 0.5*Vec(2,1,1f0)), MeshPhongMaterial(color=colorant"firebrick"))
    render(vis)
end

LoadError: Failed to precompile MeshCat [283c5d60-a78f-5afe-a0af-af636b173e11] to C:\Users\bdw19\.julia\compiled\v1.6\MeshCat\jl_CE17.tmp.

In [58]:
function show_sim(vis, qs, h)
    fps = Int(1/h)
    anim = MeshCat.Animation(fps)
    for (i,q) in enumerate(qs)
        atframe(anim, i) do
            settransform!(vis["brick"], Translation(q[1],0,q[2]))
        end
    end
    setanimation!(vis, anim)
end
if !isautograder
    show_sim(vis, h::Real) = show_sim(vis, simulate_brick(h=h)[1], h)
    show_sim(vis, 0.01);
end

LoadError: UndefVarError: vis not defined

In [59]:
using Statistics
@testset "3c" begin                  # POINTS = 10      
    @testset "simulate brick" begin  # POINTS = 8
        h = 0.01
        qans = load(resfile, "qs")
        vans = load(resfile, "vs")
        qs, vs = simulate_brick(h=h)
        eps = 1e-6

        @test [q[1]/0.01 for q in diff(qs)] ≈ [v[1] for v in vs[1:end-1]] atol=1e-6  # Sanity check velocities              POINTS = 0.5
        @test std([q[1] for q in diff(qs)]) < eps                                    # no horizontal acceleration           POINTS = 0.5
        @test all(q->q[1] > 0, diff(qs))                                             # positive horizontal velocity         POINTS = 0.5
        @test all(q->q[2] > -eps, qs)                                                # no penetration through the floor     POINTS = 1
        @test all(v->v[1] ≈ 1.0, vs)                                                 # constant horizontal velocity         POINTS = 0.5
        @test all(v->v[2] < eps, vs)                                                 # all vertical velocity is negative    POINTS = 1
        @test all(v->abs(v[2]) < eps, vs[101:end])                                   # zero vertical velocity after impact (actual impact time is before this)  # POINTS = 1
        @test qs ≈ qans atol=1e-3  # POINTS = 1.5
        @test vs ≈ vans atol=1e-3  # POINTS = 1.5
    end
end;

simulate brick: Test Failed at In[59]:16
  Expression: all((v->begin
            abs(v[2]) < eps
        end), vs[101:end])
Stacktrace:
 [1] macro expansion
   @ In[59]:16 [inlined]
 [2] macro expansion
   @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Test\src\Test.jl:1151 [inlined]
 [3] macro expansion
   @ In[59]:4 [inlined]
 [4] macro expansion
   @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Test\src\Test.jl:1151 [inlined]
 [5] top-level scope
   @ In[59]:3
simulate brick: Test Failed at In[59]:17
  Expression: ≈(qs, qans, atol = 0.001)
   Evaluated: [[0.0, 1.0], [0.01, 0.999919], [0.02, 0.999757], [0.03, 0.999514], [0.04, 0.99919], [0.05, 0.998785], [0.060000000000000005, 0.998299], [0.07, 0.9977320000000001], [0.08, 0.9970840000000001], [0.09, 0.9963550000000001]  …  [2.909999999999982, -3.279205336908196e-8], [2.9199999999999817, -3.2792053369080876e-8], [2.9299999999999815, -3.279205336908142e-8], [2.9399999999999813, -3.27920533690

LoadError: [91mSome tests did not pass: 6 passed, 3 failed, 0 errored, 0 broken.[39m

## EXTRA CREDIT: Make it fast! (max 15 pts)
You can earn extra credit by making your QP solver fast. Points will be given relative to the speed of OSQP, a state-of-the-art QP solver. There will be four different levels:
1. Less than 0.5x the time of OSQP (2x slower that OSQP) (2 pts)
2. Faster than OSQP (5 pts)
3. 2x faster than OSQP (8 pts)
4. Faster than Brian's solution (about 5x faster than OSQP) (10 pts)

It will be timed on the brick simulator. Further extra credit (5 pts) may be assigned if you implement equality constraints and show it's able to successfully solve them.

Tips:
* Check out the `StaticArrays` package
* Consider making your own solver type
* Avoid allocating new memory
* Use the `BenchmarkTools` package to check the performance of individual pieces
* Check out the [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
* Write a version of your simulation code that uses OSQP to compare performance

In [ ]:
# Sample timing results
using BenchmarkTools
println("Student solution")
@btime simulate_brick();

In [ ]:
function simulate_brick_OSQP(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0

    # Build QP
    qp = build_qp(q0, v0; mass=m, h=h)
    n,m,p = size(qp)
    g = [0,9.81]
    model = OSQP.Model()
    OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,p)], u=[qp.b; qp.d],
        eps_abs=1e-6, eps_rel=1e-6, verbose=false)

    # Simulation Loop
    for i = 1:length(times)-1
        # Update the qp with the new values
        update_qp!(qp, qs[i], vs[i])
        OSQP.update!(model, q=qp.q, u=qp.d)
        
        # Solve the QP for the next velocity
        res = OSQP.solve!(model)
        vs[i+1] .= res.x
        
        # Use backward Euler to propagate the state
        qs[i+1] .= qs[i] + h*vs[i+1]
    end
    return qs, vs
end

In [ ]:
println("OSQP Solution")
@btime simulate_brick_OSQP();